# 🔬 Simple MoS₂ Image Diagnostic Tool

## Purpose
**Bulletproof** diagnostic tool that analyzes your images and recommends the optimal threshold.

**No complex OpenCV methods** - just simple statistics and visual comparison!

## Instructions
1. Upload your images to `/content/images/` in Google Colab
2. Run all cells
3. Look at the visual results to see which threshold works best
4. Use the recommended threshold in `MoS2_UltraSensitive.ipynb`

In [ ]:
# Install and import required packages
!pip install opencv-python-headless matplotlib numpy

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
import os

# Create directories
os.makedirs('/content/images', exist_ok=True)
os.makedirs('/content/diagnostics', exist_ok=True)

print("✓ Environment setup complete")

In [ ]:
def analyze_simple(image_path):
    """Simple, bulletproof image analysis using only basic statistics"""
    print(f"\n{'='*80}")
    print(f"📸 Analyzing: {Path(image_path).name}")
    print(f"{'='*80}")
    
    # Load image
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"❌ Failed to load image")
        return None
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    
    # Basic statistics (bulletproof!)
    mean_val = float(gray.mean())
    median_val = float(np.median(gray))
    std_val = float(gray.std())
    min_val = int(gray.min())
    max_val = int(gray.max())
    
    # Percentiles (bulletproof!)
    p05 = float(np.percentile(gray, 5))
    p10 = float(np.percentile(gray, 10))
    p25 = float(np.percentile(gray, 25))
    p50 = float(np.percentile(gray, 50))
    p75 = float(np.percentile(gray, 75))
    p90 = float(np.percentile(gray, 90))
    p95 = float(np.percentile(gray, 95))
    
    print(f"\n📊 Image Statistics:")
    print(f"   Size: {img.shape[1]} x {img.shape[0]} pixels")
    print(f"   Data type: {img.dtype}")
    
    print(f"\n💡 Intensity Statistics:")
    print(f"   Mean: {mean_val:.1f}")
    print(f"   Median: {median_val:.1f}")
    print(f"   Std Dev: {std_val:.1f}")
    print(f"   Min: {min_val}")
    print(f"   Max: {max_val}")
    
    print(f"\n📈 Percentiles:")
    print(f"   5th: {p05:.1f}")
    print(f"   10th: {p10:.1f}")
    print(f"   25th: {p25:.1f}")
    print(f"   50th (median): {p50:.1f}")
    print(f"   75th: {p75:.1f}")
    print(f"   90th: {p90:.1f}")
    print(f"   95th: {p95:.1f}")
    
    # Calculate suggested thresholds (simple math, no OpenCV!)
    thresh_mean_std = mean_val - std_val
    thresh_median = median_val
    thresh_p25 = p25
    thresh_p10 = p10
    
    print(f"\n🎯 Suggested Thresholds (Simple Statistics):")
    print(f"   Mean - 1σ: {thresh_mean_std:.1f}")
    print(f"   Median (50th): {thresh_median:.1f}")
    print(f"   25th percentile: {thresh_p25:.1f}")
    print(f"   10th percentile: {thresh_p10:.1f}")
    print(f"   Current (original): 140.0")
    
    return {
        'name': Path(image_path).name,
        'img_rgb': img_rgb,
        'gray': gray,
        'stats': {
            'mean': mean_val,
            'median': median_val,
            'std': std_val,
            'min': min_val,
            'max': max_val,
            'p05': p05,
            'p10': p10,
            'p25': p25,
            'p50': p50,
            'p75': p75,
            'p90': p90,
            'p95': p95
        },
        'suggested_thresholds': {
            'mean_minus_std': thresh_mean_std,
            'median': thresh_median,
            'p25': thresh_p25,
            'p10': thresh_p10
        }
    }

In [ ]:
def test_thresholds_visual(result):
    """Visual comparison of different thresholds"""
    img_rgb = result['img_rgb']
    gray = result['gray']
    name = result['name']
    
    # Test a range of thresholds
    test_thresholds = [
        70,  # Very aggressive
        80,  # Mean - std average
        100, # Lower than median
        107, # 25th percentile average
        110, # Around median
        120, # Higher
        140, # Current (original)
        160  # Very conservative
    ]
    
    fig, axes = plt.subplots(3, 4, figsize=(24, 18))
    axes = axes.ravel()
    
    # Show original
    axes[0].imshow(img_rgb)
    axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Show grayscale
    axes[1].imshow(gray, cmap='gray')
    axes[1].set_title('Grayscale', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Show histogram
    axes[2].hist(gray.ravel(), bins=256, range=(0, 256), color='blue', alpha=0.7)
    for thresh in test_thresholds:
        axes[2].axvline(thresh, linestyle='--', alpha=0.5, linewidth=1.5)
    axes[2].axvline(140, color='red', linestyle='--', linewidth=2, label='Current (140)')
    axes[2].set_title('Intensity Histogram', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Intensity')
    axes[2].set_ylabel('Frequency')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    # Hide unused subplot
    axes[3].axis('off')
    
    # Test each threshold
    flake_counts = []
    
    for idx, threshold in enumerate(test_thresholds):
        ax_idx = idx + 4
        
        # Apply threshold
        binary = (gray < threshold).astype(np.uint8) * 255
        
        # Morphological operations (same as pipeline)
        kernel_open = np.ones((2, 2), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel_open)
        
        kernel_close = np.ones((4, 4), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel_close)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter by area (200-15000 pixels, same as pipeline)
        valid_contours = [c for c in contours if 200 <= cv2.contourArea(c) <= 15000]
        
        # Draw results
        result_img = img_rgb.copy()
        cv2.drawContours(result_img, valid_contours, -1, (0, 255, 0), 3)
        
        # Add flake count labels
        for i, contour in enumerate(valid_contours[:10]):  # Label first 10
            M = cv2.moments(contour)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                cv2.putText(result_img, str(i+1), (cx-10, cy+10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        
        axes[ax_idx].imshow(result_img)
        
        # Title with color coding
        title_color = 'green' if 3 <= len(valid_contours) <= 20 else 'red'
        if threshold == 107:
            title_color = 'blue'  # Highlight recommended
        
        title = f'Threshold: {threshold}\n{len(valid_contours)} flakes detected'
        if threshold == 140:
            title += ' (Current)'
        elif threshold == 107:
            title += ' ⭐ Recommended'
        
        axes[ax_idx].set_title(title, fontsize=12, fontweight='bold', color=title_color)
        axes[ax_idx].axis('off')
        
        flake_counts.append((threshold, len(valid_contours)))
    
    plt.suptitle(f'Threshold Comparison: {name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save figure
    output_path = f'/content/diagnostics/{Path(name).stem}_simple_diagnostic.png'
    plt.savefig(output_path, dpi=200, bbox_inches='tight')
    print(f"\n💾 Saved: {output_path}")
    plt.show()
    
    return flake_counts

In [ ]:
def recommend_threshold(all_results):
    """Analyze all images and recommend the best threshold"""
    print(f"\n{'='*80}")
    print(f"🎯 FINAL RECOMMENDATIONS")
    print(f"{'='*80}")
    
    # Aggregate statistics
    all_means = [r['stats']['mean'] for r in all_results]
    all_medians = [r['stats']['median'] for r in all_results]
    all_stds = [r['stats']['std'] for r in all_results]
    all_p25 = [r['stats']['p25'] for r in all_results]
    all_p10 = [r['stats']['p10'] for r in all_results]
    
    avg_mean = np.mean(all_means)
    avg_median = np.mean(all_medians)
    avg_std = np.mean(all_stds)
    avg_p25 = np.mean(all_p25)
    avg_p10 = np.mean(all_p10)
    
    print(f"\n📊 Statistics Across {len(all_results)} Images:")
    print(f"   Average mean intensity: {avg_mean:.1f}")
    print(f"   Average median: {avg_median:.1f}")
    print(f"   Average std deviation: {avg_std:.1f}")
    print(f"   Average 25th percentile: {avg_p25:.1f}")
    print(f"   Average 10th percentile: {avg_p10:.1f}")
    
    print(f"\n🔍 Image Assessment:")
    
    # Brightness assessment
    if avg_mean > 150:
        print(f"   ⚠️  Images are BRIGHT (mean={avg_mean:.1f})")
        brightness = "bright"
    elif avg_mean < 100:
        print(f"   ⚠️  Images are DARK (mean={avg_mean:.1f})")
        brightness = "dark"
    else:
        print(f"   ✅ Images have normal brightness (mean={avg_mean:.1f})")
        brightness = "normal"
    
    # Contrast assessment
    if avg_std < 20:
        print(f"   ⚠️  LOW CONTRAST (std={avg_std:.1f})")
        contrast = "low"
    elif avg_std > 40:
        print(f"   ✅ EXCELLENT CONTRAST (std={avg_std:.1f})")
        contrast = "high"
    else:
        print(f"   ✅ Good contrast (std={avg_std:.1f})")
        contrast = "normal"
    
    # Determine recommended threshold
    print(f"\n💡 Threshold Recommendation:")
    
    # Primary recommendation: 25th percentile
    recommended = int(round(avg_p25))
    
    print(f"\n   🌟 PRIMARY RECOMMENDATION: {recommended}")
    print(f"      (Based on 25th percentile average)")
    print(f"      This should capture flakes in the {int(avg_p10)}-{int(avg_median)} range")
    
    # Alternative recommendations
    alt1 = int(round(avg_median))
    alt2 = int(round(avg_mean - avg_std))
    alt3 = int(round(avg_p10))
    
    print(f"\n   📋 Alternative thresholds to try:")
    print(f"      • {alt1} (median - if primary gives too much noise)")
    print(f"      • {alt2} (mean-σ - more aggressive)")
    print(f"      • {alt3} (10th percentile - very aggressive)")
    
    print(f"\n   ❌ Current threshold: 140")
    if 140 > avg_median + 20:
        print(f"      This is TOO HIGH! Most flakes are being missed!")
    
    # Code snippet
    print(f"\n📝 Code Update for MoS2_UltraSensitive.ipynb:")
    print(f"\n```python")
    print(f"# In __init__ method (Cell 2, line 3), change from:")
    print(f"# self.intensity_threshold = 140")
    print(f"# To:")
    print(f"self.intensity_threshold = {recommended}  # Optimized for your images")
    print(f"```")
    
    if contrast == "low":
        print(f"\n⚠️  Also add CLAHE preprocessing for low contrast:")
        print(f"\n```python")
        print(f"# In stage1_detect_flakes, after loading image, add:")
        print(f"clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))")
        print(f"gray = clahe.apply(gray)")
        print(f"```")
    
    # Save recommendations
    recommendations = {
        'recommended_threshold': recommended,
        'alternative_thresholds': [alt1, alt2, alt3],
        'current_threshold': 140,
        'statistics': {
            'avg_mean': avg_mean,
            'avg_median': avg_median,
            'avg_std': avg_std,
            'avg_p25': avg_p25,
            'avg_p10': avg_p10
        },
        'assessment': {
            'brightness': brightness,
            'contrast': contrast,
            'needs_clahe': contrast == 'low'
        },
        'individual_images': [
            {
                'name': r['name'],
                'mean': r['stats']['mean'],
                'median': r['stats']['median'],
                'p25': r['stats']['p25']
            } for r in all_results
        ]
    }
    
    # Save to JSON
    json_path = '/content/diagnostics/simple_recommendations.json'
    with open(json_path, 'w') as f:
        json.dump(recommendations, f, indent=2)
    
    print(f"\n💾 Saved recommendations to: {json_path}")
    
    return recommendations

In [ ]:
# Main analysis
print(f"\n{'='*80}")
print(f"🔬 Simple MoS2 Diagnostic Tool")
print(f"{'='*80}")

# Find images
image_dir = Path('/content/images')
image_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.tif', '.bmp']

image_files = []
for ext in image_extensions:
    image_files.extend(image_dir.glob(f'*{ext}'))
    image_files.extend(image_dir.glob(f'*{ext.upper()}'))

image_files = sorted(image_files)

print(f"\nFound {len(image_files)} images:")
for img in image_files:
    print(f"   • {img.name}")

if not image_files:
    print("\n❌ No images found in /content/images/")
    print("Please upload your images and run this cell again.")
else:
    # Analyze each image
    all_results = []
    all_flake_counts = {}
    
    for img_path in image_files:
        # Simple analysis
        result = analyze_simple(str(img_path))
        if result:
            all_results.append(result)
            
            # Visual threshold testing
            print(f"\n📊 Testing different thresholds visually...")
            flake_counts = test_thresholds_visual(result)
            all_flake_counts[result['name']] = flake_counts
            
            # Print summary for this image
            print(f"\n📈 Flake Count Summary for {result['name']}:")
            for thresh, count in flake_counts:
                marker = "⭐" if thresh == 107 else ("❌" if thresh == 140 else "")
                print(f"   Threshold {thresh:3d}: {count:2d} flakes {marker}")
    
    if all_results:
        # Generate final recommendations
        recommendations = recommend_threshold(all_results)
        
        print(f"\n{'='*80}")
        print(f"✅ Diagnostic Complete!")
        print(f"{'='*80}")
        print(f"\n📁 Results saved to: /content/diagnostics/")
        print(f"\n🎯 Next steps:")
        print(f"   1. Review the visual comparisons above")
        print(f"   2. Use recommended threshold: {recommendations['recommended_threshold']}")
        print(f"   3. Update MoS2_UltraSensitive.ipynb")
        print(f"   4. Run the full pipeline!")
    else:
        print(f"\n❌ No images were successfully analyzed")